In [ ]:
import importlib

import dataset, metrics, plotting, config, network
from models import cgan_model
import numpy as np
import random

importlib.reload(network)
importlib.reload(dataset)
importlib.reload(metrics)
importlib.reload(plotting)
importlib.reload(config)
importlib.reload(cgan_model)

In [ ]:
import os

dataset_config = config.DatasetConfig(scenario="CA-housing-single")

assert(dataset_config.scenario == "CA-housing"
       or dataset_config.scenario == "ailerons"
       or dataset_config.scenario == "CA-housing-single")
fig_dir = f"../figures/{dataset_config.scenario}"

try:
    os.mkdir(fig_dir)
    print(f"Directory {fig_dir} created ") 
except FileExistsError:
    print(f"Directory {fig_dir} already exists replacing files in this notebook")

In [ ]:
if dataset_config.scenario == "CA-housing" or dataset_config.scenario == "CA-housing-single":
    exp_config = config.Config(
        model=config.ModelConfig(activation="elu", lr_gen=0.0001, lr_disc=0.001, dec_gen=0, dec_disc=0, 
                                 optim_gen="Adam", optim_disc="Adam", z_input_size=1),
        training=config.TrainingConfig(n_epochs=500, batch_size=100, n_samples=50),
        dataset=dataset_config,
        run=config.RunConfig(save_fig=1)
    )
    
elif dataset_config.scenario == "ailerons":
    exp_config = config.Config(
        model=config.ModelConfig(activation="elu", lr_gen=0.0001, lr_disc=0.0005, dec_gen=0, dec_disc=0, 
                                 optim_gen="Adam", optim_disc="Adam", z_input_size=1),
        training=config.TrainingConfig(n_epochs=500, batch_size=100, n_samples=50),
        dataset=dataset_config,
        run=config.RunConfig(save_fig=1)
    )

In [ ]:
# Set random seed
np.random.seed(exp_config.model.random_seed)
random.seed(exp_config.model.random_seed)

from tensorflow import set_random_seed
set_random_seed(exp_config.model.random_seed)

In [ ]:
X_train, y_train, X_test, y_test, X_valid, y_valid = dataset.get_dataset(scenario=exp_config.dataset.scenario, 
                                                                         seed=exp_config.model.random_seed)

In [ ]:
X_train.shape, len(X_valid), len(X_test)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# GAN

In [ ]:
cgan = cgan_model.CGAN(exp_config)
d_loss_err, d_loss_true, d_loss_fake, g_loss_err, g_pred, g_true = cgan.train(X_train_scaled, y_train, 
                                                                              epochs=exp_config.training.n_epochs,
                                                                              batch_size=exp_config.training.batch_size)

ypred_gan_test = cgan.predict(X_test_scaled)

In [ ]:
plotting.plots(d_loss_err, d_loss_true, d_loss_fake, g_loss_err, g_pred, g_true, fig_dir, exp_config.run.save_fig)

In [ ]:
ypred_mean_gan_test, ypred_median_gan_test, ypred_gan_sample_test = cgan.sample(X_test_scaled, 
                                                                                exp_config.training.n_samples)

In [ ]:
ypred_mean_gan_train, ypred_median_gan_train, ypred_gan_sample_train = cgan.sample(X_train_scaled, 
                                                                                   exp_config.training.n_samples)

# NN

In [ ]:
import keras

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])
model.compile(loss="mean_absolute_error", optimizer="sgd")

In [ ]:
callbacks = [keras.callbacks.EarlyStopping(patience=10)]
history = model.fit(X_train_scaled, y_train,
                    validation_data=(X_test_scaled, y_test), epochs=100,
                    callbacks=callbacks)

In [ ]:
plotting.plot_learning_curves(history)

In [ ]:
ypred_nn_test = model.predict(X_test_scaled)

# GP

In [ ]:
import GPy

noise = 1
length = 1

run_hyperopt_search = False

kernel = GPy.kern.RBF(input_dim=cgan.x_input_size, variance=noise, lengthscale=length)
gpr = GPy.models.GPRegression(X_train_scaled, y_train.reshape(-1, 1), kernel)

if run_hyperopt_search:
    gpr.optimize(messages=True)

In [ ]:
ypred_gp_test, cov_test = gpr.predict(X_test_scaled)

# Plots

In [ ]:
# plotting.plot_densities_joint(y_test, ypred_nn_test, ypred_mean_gan_test, ypred_gp_test, 
#                              "Linear-vs-GAN-vs-GP P(y) density", fig_dir=fig_dir, 
#                              prefix="all_marginalized", save_fig=exp_config.run.save_fig, at_x=True)

plotting.plot_datadistrib_joint(y_test, ypred_nn_test, ypred_mean_gan_test, ypred_gp_test, 
                                "Linear-vs-GAN-vs-GP P(y) density", fig_dir=fig_dir, 
                                prefix="all_marginalized", save_fig=exp_config.run.save_fig)

# Metrics

In [ ]:
n_eval_runs = 10

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mse_gan_= []
mae_gan_ = []
for i in range(n_eval_runs):
    ypred_mean_gan_test_, ypred_median_gan_test_, _ = cgan.sample(X_test_scaled, exp_config.training.n_samples)
    mae_gan_.append(mean_absolute_error(y_test, ypred_median_gan_test_))
    mse_gan_.append(mean_squared_error(y_test, ypred_mean_gan_test_))

In [ ]:
nn_mae = mean_absolute_error(ypred_nn_test, y_test)
gp_mae = mean_absolute_error(ypred_gp_test, y_test)
gan_mae_mean = np.mean(np.asarray(mae_gan_))
gan_mae_std = np.std(np.asarray(mae_gan_))

print(f"NN MAE test: {nn_mae}")
print(f"GP MAE test: {gp_mae}")
print(f"GAN MAE test: {gan_mae_mean} +- {gan_mae_std}")

In [ ]:
nn_mse = mean_squared_error(ypred_nn_test, y_test)
gp_mse = mean_squared_error(ypred_gp_test, y_test)
gan_mse_mean = np.mean(np.asarray(mse_gan_))
gan_mse_std = np.std(np.asarray(mse_gan_))

print(f"NN MSE test: {nn_mse}")
print(f"GP MSE test: {gp_mse}")
print(f"GAN MSE test: {gan_mse_mean} +- {gan_mse_std}")

In [ ]:
nn_nlpd = metrics.gaussian_NLPD(y_test, ypred_nn_test, np.ones(len(ypred_nn_test)), "NN")

In [ ]:
gp_nlpd = metrics.gaussian_NLPD(y_test, ypred_gp_test, cov_test, "GP")

In [ ]:
gan_nlpd_train, w, lls = metrics.Parzen(cgan, X_valid_scaled, y_valid, n_sample=exp_config.training.n_samples)
nlpd_ = []
for i in range(n_eval_runs):
    nlpd_.append(metrics.Parzen_test(cgan, X_test_scaled, y_test, w, exp_config.training.n_samples))
gan_nlpd_test = np.mean(nlpd_)
gan_nlpd_std_test = np.std(nlpd_)
print(f"GAN Train NLLH: {gan_nlpd_train}")
print(f"GAN Test NLLH: mean {gan_nlpd_test} std {gan_nlpd_std_test}")

In [ ]:
if exp_config.run.save_fig:
    file = open(f"{fig_dir}/metrics.txt","w") 

    file.write(f"===Test MAE===\n")
    file.write(f"NN MAE test: {nn_mae}\n") 
    file.write(f"GP MAE test: {gp_mae}\n")
    file.write(f"GAN MAE test: {gan_mae_mean} +- {gan_mae_std}\n")
    file.write(f"===Test MSE===\n")
    file.write(f"NN MSE test: {nn_mse}\n")
    file.write(f"GP MSE test: {gp_mse}\n")
    file.write(f"GAN MSE test: {gan_mse_mean} +- {gan_mse_std}\n")
    file.write(f"===Test NLPD===\n")
    file.write(f"NN Gaussian NLPD: {nn_nlpd}\n")
    file.write(f"GP Gaussian NLPD: {gp_nlpd}\n")
    file.write(f"GAN NLPD: {gan_nlpd_test} +- {gan_nlpd_std_test}\n")
    file.close() 